In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# DirPath = ('/content/drive/My Drive/')

In [3]:

!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
!pip install -q -U accelerate
!pip install -q -U datasets

ERROR: Could not find a version that satisfies the requirement bitsandbytes (from versions: none)
ERROR: No matching distribution found for bitsandbytes


In [4]:
!pip install -q transformers
!pip install -q peft accelerate bitsandbytes safetensors

ERROR: Could not find a version that satisfies the requirement peft (from versions: none)
ERROR: No matching distribution found for peft


In [ ]:
!pip freeze > kaggle_requirements.txt


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification, IntervalStrategy, BloomForSequenceClassification, EarlyStoppingCallback

In [ ]:
model_name = "bigscience/bloom-560m"#"bigscience/bloom-3b"#"bigscience/bloom-560m"

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
import wandb

wandb.login(key="e0b5e8662a12704a23b8b8dc633d1d688941df16")

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0})
model = BloomForSequenceClassification.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0},num_labels=2)

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"#"CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
# from datasets import load_dataset
# data = load_dataset("Abirate/english_quotes")
# data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [ ]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/banglafakenews/train.csv')[ ['Headline','Content','Label'] ]
val_df = pd.read_csv('/kaggle/input/banglafakenews/val.csv')[ ['Headline','Content','Label']]

In [ ]:
# from sklearn.model_selection import train_test_split
HCSep = " \\\\ "
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['Headline'])):
      text_list.append(dataset['Headline'][i]+HCSep+dataset['Content'][i])
      label_list.append(int(dataset['Label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [ ]:
max_length=512
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)

In [ ]:
# print(train_dataset[:2])
# model(input_ids=train_dataset[:1]['input_ids'],attention_mask=train_dataset[:1]['attention_mask'])

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    print("\nAfter iteration multiple of 2514\n")
    print({
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1": f1
    })
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    # train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        save_total_limit = 1,
        warmup_steps=2,
        # max_steps=20,
        num_train_epochs=2,
        learning_rate=8e-5,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        metric_for_best_model = 'f1',
        evaluation_strategy = IntervalStrategy.STEPS,
        weight_decay=0.005,
        eval_steps = 2694,
        logging_steps=2694,
        save_steps = 2694,
        load_best_model_at_end=True,
    ),
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer.train()

In [ ]:
from sklearn.metrics import classification_report

test_df = pd.read_csv('/kaggle/input/banglafakenews/test.csv')[ ['Headline','Content','Label']]
test_texts, test_labels = pandaToList(test_df)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)
test_dataset = ViolenceDataset(test_encodings, test_labels)

predictions = trainer.predict(test_dataset)

# Get predicted labels and true labels
predicted_labels = predictions.predictions.argmax(axis=1)
true_labels = predictions.label_ids

# Generate the classification report
class_report = classification_report(true_labels, predicted_labels)

print(class_report)

In [ ]:
# text = "Ask not what your country"
# device = "cuda:0"
# inputs = tokenizer(text, return_tensors="pt").to(device)

# print(inputs)

# # outputs = model.generate(**inputs, max_new_tokens=20)
# outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))